<a href="https://colab.research.google.com/github/dsinha23/dsinha23/blob/main/default_time_simulation/Monte_carlo_asymp_exact_II_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.stats import nct, ncx2
from scipy.special import hyp1f1
import sys

In [2]:
def lambda_max_generator(eps, y, theta, sigma, kappa):

  # Define lambda function for root finding
  G1 = lambda H : hyp1f1(H/kappa, (2 * kappa * theta)/(sigma**2), (2 * kappa * y)/(sigma**2))/hyp1f1(H/kappa, \
    (2 * kappa * theta)/(sigma**2), (2*kappa*H)/(sigma**2)) - eps
  F1 = lambda H : (hyp1f1(H/kappa, (2 * kappa * theta)/(sigma**2), (2 * kappa * y)/(sigma**2))/hyp1f1(H/kappa, \
    (2 * kappa * theta)/(sigma**2), (2*kappa*H)/(sigma**2)) - eps)**2
  # Root finding
  Hstar = scipy.optimize.minimize(F1, x0 = y,method='L-BFGS-B', bounds=[(y, None)]).x[0]
  while G1(Hstar) > 0:
    Hstar = Hstar + 1e-4
  return Hstar

In [15]:
def simulate_cir_jump(x0, kappa, theta, sigma, T, delta, eps, lambda_benchmark, low=0.24, high=0.96):

    events = []
    marks = []
    t = 0.0
    lambda_t = x0

    time_grid = [t]
    lambda_path = [lambda_t]
    gamma = theta

    count = 0

    while t < T:
        # Compute CIR parameters for exact simulation over time tau

        if lambda_t <= gamma:
          lambda_max = lambda_benchmark
        else:
          lambda_max = lambda_t + lambda_benchmark - gamma
        tau = np.random.exponential(1 / lambda_max)
        t_proposed = t + tau
        if t_proposed > T:
            break

        # Exact CIR transition over tau

        c = sigma**2 * (1 - np.exp(-kappa * tau)) / (4 * kappa)
        df = 4 * kappa * theta / sigma**2
        nc = 4 * kappa * lambda_t * np.exp(-kappa * tau) / (sigma**2 * (1 - np.exp(-kappa * tau)))


    # Sample from non-central chi-squared
        lambda_proposed = c * ncx2.rvs(df, nc)

        # Accept or reject via thinning
        u = np.random.uniform(0,1)
        if u <= np.minimum(lambda_proposed / lambda_max,1):
            # Accept: register event and mark
            t = t_proposed
            events.append(t)
            mark = np.random.choice([low,high])
            marks.append(mark)

            # Intensity jumps up
            lambda_t = lambda_proposed + delta * mark
        else:
            # Reject: update time
            t = t_proposed
            lambda_t = lambda_proposed

        time_grid.append(t)
        lambda_path.append(lambda_t)

    return np.array(events), np.array(marks), np.array(time_grid), np.array(lambda_path)

In [14]:
num_trials = 30000
TrueValue = 12.1708

x0 = 0.7 # initial intensity
kappa = 2.62
theta = 1.61
sigma = 0.62
T = 5.0
delta = 2.99
L_T = np.zeros(num_trials)
default_time  = np.array([])
eps = 1/num_trials
lambda_benchmark = lambda_max_generator(eps, theta, theta, sigma, kappa)
for i in range(num_trials):
  sys.stdout.write(f"\rTrial No.: {i+1}")
  sys.stdout.flush()
  events, marks, time, lambda_path = simulate_cir_jump(x0, kappa, theta, sigma, T, delta, eps,lambda_benchmark)
  L_T[i] = np.sum(marks)
  default_time = np.append(default_time, time)
bias = np.mean(L_T) - TrueValue
std = np.std(L_T)/np.sqrt(num_trials)
print('\nBias:', bias)
print('Standard deviation:', std)
print('Expected loss:', np.mean(L_T))
print('RMSE:',np.sqrt(bias**2 + std**2))

Trial No.: 30000
Bias: -0.014088000000000989
Standard deviation: 0.04857138452115471
Expected loss: 12.156711999999999
RMSE: 0.050573225508186596
